In [1]:
! pip3 install -U scikit-learn scipy matplotlib
! pip3 install -U shap 
! pip3 install -U lime 
! pip3 install -U xgboost 
! pip3 install -U eli5 
! pip3 install -U seaborn 

! pip3 install ipywidgets

! pip3 install lightgbm

! pip3 install mlxtend

Requirement already up-to-date: scikit-learn in /home/ava/.local/lib/python3.8/site-packages (0.24.1)
Requirement already up-to-date: scipy in /home/ava/.local/lib/python3.8/site-packages (1.6.0)
Requirement already up-to-date: matplotlib in /home/ava/.local/lib/python3.8/site-packages (3.3.4)
Requirement already up-to-date: shap in /home/ava/.local/lib/python3.8/site-packages (0.38.1)
Requirement already up-to-date: lime in /home/ava/.local/lib/python3.8/site-packages (0.2.0.1)
Requirement already up-to-date: xgboost in /home/ava/.local/lib/python3.8/site-packages (1.3.3)
Requirement already up-to-date: eli5 in /home/ava/.local/lib/python3.8/site-packages (0.11.0)
Requirement already up-to-date: seaborn in /home/ava/.local/lib/python3.8/site-packages (0.11.1)


In [2]:
import os

import numpy as np 
import pandas as pd 

import pickle

import itertools 

from eli5 import show_prediction

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb 
import lightgbm as lgb

import seaborn as sns 

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt 

from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import EnsembleVoteClassifier

import sys
sys.path.append('..')

from inxai import *
import shap
import lime

In [3]:
path = '../examples/'

def to_pickle(obj, name):
    with open(path + name + '.pickle', 'wb') as f:
        pickle.dump(obj, f)

def from_pickle(name):
    return pd.read_pickle(path + name + '.pickle')


In [4]:
df = pd.read_csv("../examples/data/compass/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv")
print(df.shape)
display(df.columns)
df.head()

(6172, 12)


Index(['Two_yr_Recidivism', 'Number_of_Priors', 'score_factor',
       'Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'African_American',
       'Asian', 'Hispanic', 'Native_American', 'Other', 'Female',
       'Misdemeanor'],
      dtype='object')

,Two_yr_Recidivism,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
0,0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,1,4,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1
4,1,14,1,0,0,0,0,0,0,0,0,0


In [5]:
TARGET_COL = "Two_yr_Recidivism"

X = df.drop([TARGET_COL],axis=1)
y = df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.33, random_state=42)

print("TRAIN:")
print(X_train.head())
print("\nTEST:")
print(y_test.head())

TRAIN:
      Number_of_Priors  score_factor  Age_Above_FourtyFive  \
5025                 3             0                     0   
3098                 2             0                     0   
1187                 3             1                     0   
5238                 3             1                     1   
4270                 1             0                     1   

      Age_Below_TwentyFive  African_American  Asian  Hispanic  \
5025                     1                 0      0         0   
3098                     1                 1      0         0   
1187                     0                 1      0         0   
5238                     0                 0      0         1   
4270                     0                 1      0         0   

      Native_American  Other  Female  Misdemeanor  
5025                0      0       0            1  
3098                0      0       1            0  
1187                0      0       0            0  
5238                0

## Modele 

In [6]:
svc_radial = SVC(kernel='rbf',probability=True) # does not work with eli

svc_radial.fit(X_train, y_train)
svc_radial_preds = svc_radial.predict(X_test)

# to_pickle(svc_radial, 'model_svc_radial')

print(accuracy_score(y_test, svc_radial_preds))
print(classification_report(y_test, svc_radial_preds))

# show_prediction(svc_radial, X_train.iloc[1], feature_names = X.columns.tolist(), 
#                 show_feature_values=True)

0.658321060382916
              precision    recall  f1-score   support

           0       0.68      0.71      0.69      1118
           1       0.63      0.60      0.61       919

    accuracy                           0.66      2037
   macro avg       0.65      0.65      0.65      2037
weighted avg       0.66      0.66      0.66      2037



In [7]:
svc_lin = SVC(kernel='linear',probability=True) 
svc_lin.fit(X_train, y_train)
svc_lin_preds = svc_lin.predict(X_test)

# to_pickle(svc_lin, 'model_svc_lin')

print(accuracy_score(y_test, svc_lin_preds))
print(classification_report(y_test, svc_lin_preds))

show_prediction(svc_lin, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

0.6514482081492391
              precision    recall  f1-score   support

           0       0.68      0.68      0.68      1118
           1       0.61      0.61      0.61       919

    accuracy                           0.65      2037
   macro avg       0.65      0.65      0.65      2037
weighted avg       0.65      0.65      0.65      2037



In [8]:
xgbc = xgb.XGBClassifier() 
xgbc.fit(X_train, y_train)
xgbc_preds = xgbc.predict(X_test)

# to_pickle(xgbc, 'model_xgbc')

print(accuracy_score(y_test, xgbc_preds))
print(classification_report(y_test, xgbc_preds))

show_prediction(xgbc, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

[19:39:16] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6666666666666666
              precision    recall  f1-score   support

           0       0.69      0.73      0.70      1118
           1       0.64      0.60      0.62       919

    accuracy                           0.67      2037
   macro avg       0.66      0.66      0.66      2037
weighted avg       0.67      0.67      0.67      2037



The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


In [9]:
params = {'metric' : 'auc',
          'boosting_type' : 'gbdt',
          'colsample_bytree' : 0.92,
#           'max_depth' : -1,
#           'n_estimators' : 200,
          'min_child_samples': 4, 
#           'min_child_weight': 0.1,
          'subsample': 0.85,
#           'verbose' : -1,
          'num_threads' : 4 }

lgbm = lgb.train(params,                                    # does not work with eli
                 lgb.Dataset(X_train,label=y_train),
#                  2500,
                 valid_sets=lgb.Dataset(X_test,label=y_test),
                 early_stopping_rounds= 15,
                 verbose_eval= 30
                 )

lgbm_preds = lgbm.predict(X_test)
lgbm_preds = (lgbm_preds > 0.5) * 1

# to_pickle(lgbm, 'model_lgbm')

print(accuracy_score(y_test, lgbm_preds))
print(classification_report(y_test, lgbm_preds))

# show_prediction(lgbm, X_train.iloc[1], feature_names = X.columns.tolist(), #estimator is not supported
#                 show_feature_values=True)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000304 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 4135, number of used features: 11
[LightGBM] [Info] Start training from score 0.457074
Training until validation scores don't improve for 15 rounds
Early stopping, best iteration is:
[7]	valid_0's auc: 0.720958
0.678939617083947
              precision    recall  f1-score   support

           0       0.68      0.80      0.73      1118
           1       0.68      0.54      0.60       919

    accuracy                           0.68      2037
   macro avg       0.68      0.67      0.67      2037
weighted avg       0.68      0.68      0.67      2037



In [10]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_preds = rfc.predict(X_test)

# to_pickle(rfc, 'model_rfc')

print(accuracy_score(y_test, rfc_preds))
print(classification_report(y_test, rfc_preds))

show_prediction(rfc, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

0.6632302405498282
              precision    recall  f1-score   support

           0       0.68      0.72      0.70      1118
           1       0.64      0.59      0.61       919

    accuracy                           0.66      2037
   macro avg       0.66      0.66      0.66      2037
weighted avg       0.66      0.66      0.66      2037



## SHAP's

In [11]:
shap_xgbc = from_pickle('shap_xgbc')


In [12]:
gm = GlobalFeatureMetric()

In [13]:
# shap_xgbc = generate_per_instance_importances(models=xgbc, X=X_test, y=y_test, framework='kernel_shap')
# to_pickle(shap_xgbc, 'shap_xgbc')
len(shap_xgbc)

2037

In [34]:
# lime_xgbc = generate_per_instance_importances(models=xgbc, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_xgbc, 'lime_xgbc')
# lime_xgbc

In [37]:
X_test[1:10]

,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
1844,1,0,0,0,0,0,1,0,0,0,1
3311,2,0,0,0,1,0,0,0,0,0,0
346,0,0,0,1,0,0,1,0,0,0,1
1728,1,1,0,1,0,0,0,0,0,1,0
2072,1,0,0,0,1,0,0,0,0,1,0
6120,3,0,0,0,0,0,0,0,0,0,1
4418,8,1,0,0,0,0,0,0,1,0,0
4119,2,1,0,0,1,0,0,0,0,0,0
3403,3,1,0,0,0,0,0,0,0,1,1


In [38]:
y_test[1:10]

1844    0
3311    0
346     0
1728    0
2072    0
6120    1
4418    1
4119    1
3403    0
Name: Two_yr_Recidivism, dtype: int64

In [39]:
np.array_split(y_test[1:10], 3)

[1844    0
 3311    0
 346     0
 Name: Two_yr_Recidivism, dtype: int64,
 1728    0
 2072    0
 6120    1
 Name: Two_yr_Recidivism, dtype: int64,
 4418    1
 4119    1
 3403    0
 Name: Two_yr_Recidivism, dtype: int64]

In [40]:
pd.DataFrame(shap_xgbc)

,0,1,2,3,4,5,6,7,8,9,10
0,0.302015,0.067122,0.029087,-0.030196,0.051102,0.000486,0.009062,0.000176,0.000947,0.015168,-0.019243
1,0.068505,0.053653,-0.008948,0.040740,0.007332,-0.000293,0.024935,-0.000272,-0.000541,-0.012110,0.025806
2,-0.013949,0.060538,-0.018786,0.055841,-0.015909,-0.000010,0.001538,-0.000318,0.001685,-0.009686,-0.014266
3,0.105171,0.044161,-0.007458,-0.174526,0.021152,-0.000871,-0.034856,-0.000262,-0.001993,-0.038439,0.024678
4,0.084181,-0.101717,-0.009765,-0.016849,0.042901,-0.000069,0.001516,-0.000281,-0.003416,0.146125,-0.010858
...,...,...,...,...,...,...,...,...,...,...,...
2032,0.206211,0.047852,-0.011734,0.011410,-0.002071,-0.000250,-0.000667,-0.000224,0.054585,0.084375,-0.007474
2033,0.088101,0.078130,0.010151,-0.040857,0.009729,0.000234,-0.002820,0.000356,0.001090,0.019450,-0.053581
2034,0.163041,0.088132,0.006301,0.028499,0.014530,0.084547,0.000462,-0.000167,-0.000569,-0.012353,0.036950
2035,0.174440,0.029509,-0.013946,0.023294,-0.003132,-0.000407,0.000551,-0.000226,-0.001201,0.085253,0.024510


In [51]:
df.loc[:, X_test.columns]
df.loc[:, TARGET_COL]

0       0
1       1
2       1
3       0
4       1
       ..
6167    0
6168    0
6169    0
6170    0
6171    1
Name: Two_yr_Recidivism, Length: 6172, dtype: int64

In [52]:
X_test.iloc[0:10, ]

,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
2292,13,1,0,0,1,0,0,0,0,0,1
1844,1,0,0,0,0,0,1,0,0,0,1
3311,2,0,0,0,1,0,0,0,0,0,0
346,0,0,0,1,0,0,1,0,0,0,1
1728,1,1,0,1,0,0,0,0,0,1,0
2072,1,0,0,0,1,0,0,0,0,1,0
6120,3,0,0,0,0,0,0,0,0,0,1
4418,8,1,0,0,0,0,0,0,1,0,0
4119,2,1,0,0,1,0,0,0,0,0,0
3403,3,1,0,0,0,0,0,0,0,1,1


In [55]:
generate_per_instance_importances(models=xgbc, X=X_test.iloc[0:30, ], y=y_test.iloc[0:30, ], framework='kernel_shap')

  0%|          | 0/30 [00:00<?, ?it/s]

[array([ 0.28583407,  0.06665211,  0.02496806, -0.04049941,  0.04785486,
         0.        ,  0.01436893,  0.        , -0.00062455,  0.01761825,
        -0.00956544]),
 array([ 0.08240228,  0.05558434, -0.0043969 ,  0.05089482,  0.0050106 ,
         0.        ,  0.02535746,  0.        ,  0.00026233, -0.01680848,
         0.01961797]),
 array([-0.01259889,  0.05928401, -0.01493309,  0.06448521, -0.01518272,
         0.        ,  0.00708879,  0.        ,  0.00264858, -0.01416926,
        -0.01082624]),
 array([ 0.11380941,  0.04737021, -0.00413159, -0.16340491,  0.02322928,
         0.        , -0.02653886,  0.        , -0.00133925, -0.05103774,
         0.01792018]),
 array([ 0.100696  , -0.10092383, -0.00662654, -0.01834325,  0.04875323,
         0.        ,  0.00721691,  0.        , -0.00270972,  0.12884923,
        -0.0060256 ]),
 array([ 0.10827121,  0.06924715, -0.00429037,  0.0265671 ,  0.00504313,
         0.        ,  0.00409026,  0.        , -0.00175083,  0.05064266,
        -

In [80]:
X_test.iloc[0:30, ].shape[0]

30

In [42]:
from functools import partial


In [ ]:
from multiprocessing import  Pool
from functools import partial

def generate_per_instance_importances_chunk(Xy_chunk, models, X_col, y_col, framework):
    X_test = Xy_chunk.loc[:, X_col]
    y_test = Xy_chunk.loc[:, y_col]
    
    print('\nWe have rows X_test: ', X_test.shape[0])
    print('\nWe have colums X_test: ', X_test.shape[1])
#     print('\nWe have rows y_test: ', y_test.shape[0])
#     print('\nWe have colums y_test: ', len(y_test))
#     print(models)
#     print(X_col)
#     print(y_col)
#     print(framework)

    array = generate_per_instance_importances(models, X_test, y_test, framework)

    print('\nDONE')
    
    return pd.DataFrame(array)
    

def generate_per_instance_importances_parallel(models, X, y, framework, n_cores = 32):
    
    Xy = pd.merge(X, y, left_index=True, right_index=True)
    Xy_split = np.array_split(Xy, n_cores)
    
    pool = Pool(n_cores)
    func_partial = partial(generate_per_instance_importances_chunk, models=models, X_col=X.columns, y_col=TARGET_COL, framework=framework)
    
    df_result = pd.concat(pool.map(func_partial, Xy_split))
    
    return df_result

# generate_per_instance_importances_chunk(pd.merge(X_test.iloc[0:30, ], y_test.iloc[0:30, ], left_index=True, right_index=True), xgbc, X_test.columns, TARGET_COL, 'kernel_shap')
generate_per_instance_importances_parallel(models=xgbc, X=X_test.iloc[0:30, ], y=y_test.iloc[0:30, ], framework='kernel_shap', n_cores = 3)


In [58]:
from multiprocessing import  Pool

def generate_per_instance_importances_parallel(models, X, y, framework, n_cores = 32):
    
    Xy = pd.merge(X, y, left_index=True, right_index=True)
    Xy_split = np.array_split(Xy, n_cores)
    
    def func(Xy_chunk):
        X_test = Xy.loc[:, X.columns]
        y_test = Xy.loc[:, TARGET_COL]
        
        array = generate_per_instance_importances(models=models, X=X_test, y=y_test, framework=framework)

        return pd.DataFrame(array)
    
    pool = Pool(n_cores)
    df_result = pd.concat(pool.map(func, Xy_split))
    
    return df_result

generate_per_instance_importances_parallel(models=xgbc, X=X_test.iloc[0:30, ], y=y_test.iloc[0:30, ], framework='kernel_shap')

AttributeError: Can't pickle local object 'generate_per_instance_importances_parallel.<locals>.func'